
启动数据库服务器
```bash
bash /mnt/petrelfs/songmingyang/code/mm/robustLMM/robustlmm/analysis/longTail/language_level/db/start_neo4j.sh
```

shell启动 odbc
```bash
cypher-shell  -a neo4j://10.140.54.10:7687  -u neo4j -p 12345678
```
数据库操作
```sql
CREATE DATABASE XXX;
:use XXX;
CREATE DATABASE lcs558kPos;
CREATE DATABASE instructmix665kPos;
```

数据操作
```sql
CREATE CONSTRAINT FOR (n:WORD) REQUIRE n.lemma IS UNIQUE;

MERGE (n:WORD { lemma: 'test' })
ON CREATE SET n.count = 1
ON MATCH SET n.count = n.count + 1;


MERGE (a:WORD { lemma:'test' })
ON CREATE SET a.count = 1
MERGE (b:WORD { lemma:'test1' })
ON CREATE SET b.count = 1
MERGE (a)-[r:CO_OCCURRENCE]->(b)
ON CREATE SET r.count = 0
WITH a,b,r
SET r.count = r.count + 1;


```
常用查询

```sql

MATCH (n) DETACH DELETE n;
MATCH (n) RETURN n;
MATCH (n) RETURN COUNT(n);
MATCH (a)-[r:CO_OCCURRENCE]->(b) RETURN r;
```

In [4]:
from neo4j import GraphDatabase
import psycopg2

uri = "neo4j://10.140.54.12:7687"
db_name="instructmix665kllama"
chart_length=20

In [5]:
# Test connection to Neo4j

driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
gds_version_query = "call dbms.components() yield name, versions, edition unwind versions as version return name, version, edition;"
with driver.session() as session:
    results = session.run(gds_version_query).data()
    print ("Neo4j Kernel version :" + str(results[0]['version']))

Neo4j Kernel version :5.19.0


### 先画出节点的top20

In [17]:
# token_level
token_level = 1

lcs="lcs558k"
lcs_dedup = "lcs558k_deduplicate"
pope="pope"
pope_dedup = "pope_deduplicate"
mme="mme"
mme_dedup = "mme_deduplicate"
instruct = "instructmix665k"
instruct_dedup = "instructmix665k_deduplicate"
db_dict=dict(database='language_level_word_count',user='songmingyang',password='123456',host='SH-IDCA1404-10-140-54-7',port='5432')

def connect_db(db_dict):
        conn = psycopg2.connect(**db_dict)
        cursor = conn.cursor()
        return conn,cursor
    
def fetch_table_data(table_name):
        conn,cursor = connect_db(db_dict)
        # cursor.execute(f"SELECT * FROM {table_name} ORDER BY count DESC;")
        cursor.execute(f"SELECT * FROM {table_name} ORDER BY count ASC;")
        data = cursor.fetchall()
        conn.close()
        return data
    

instruct_mix_data = fetch_table_data(instruct)
instruct_mix_data = [item for item in instruct_mix_data if item[1] >= 10]
token_level_data = instruct_mix_data[:20]

In [18]:
# vision level 
query = "match (n) return n;"
visual_db_name = "instructmix665kvisualobj"
# query = "match (a)-[r:CO_OCCURRENCE]->(b) return a.lemma, r.count, b.lemma;"
with driver.session(database=visual_db_name) as session:
    results = session.run(query).data()
# results.sort(key=lambda x: x['n']['count'], reverse=True)
results.sort(key=lambda x: x['n']['count'],)
vision_level_data_before = [(x['n']['lemma'], x['n']['count']) for x in results]
vision_level_data = []
while len(vision_level_data) < chart_length:
    item = vision_level_data_before.pop(0)
    if item[1]<10:
        continue
    if item[0] not in [x[0] for x in vision_level_data]+ ['',' ']:
        vision_level_data.append(item)



In [19]:
# what_word level
from mhr.utils.utils import process_jsonl
what_word_file = "/mnt/petrelfs/songmingyang/songmingyang/data/llava_train/LLaVA-Instruct-150K/llava_v1_5_mix665k_what_word_llama.jsonl"
what_word_data = process_jsonl(what_word_file)
word_cnt_list = []
for item in what_word_data:
    what_words = item['objects'].strip().split(',')
    word_cnt_list.extend(what_words)
word_cnt_dict = {}
for word in word_cnt_list:
    if word in word_cnt_dict:
        word_cnt_dict[word] += 1
    else:
        word_cnt_dict[word] = 1

word_cnt_list = [(k,v) for k,v in word_cnt_dict.items()]
# word_cnt_list.sort(key=lambda x:x[1],reverse=True)
word_cnt_list.sort(key=lambda x:x[1],)
what_word_data = []
while len(what_word_data) < chart_length:
    item = word_cnt_list.pop(0)
    if item[1]<10:
        continue
    if item[0] not in ['',' ']+[x[0] for x in what_word_data]:
        what_word_data.append(item)

In [20]:
# co occurrence level
# query = "match (n) return n;"
visual_db_name = "instructmix665kvisualobj"
query = "match (a)-[r:CO_OCCURRENCE]->(b) return a.lemma, r.count, b.lemma;"
with driver.session(database=visual_db_name) as session:
    results = session.run(query).data()
# results.sort(key=lambda x: x['r.count'], reverse=True)
results.sort(key=lambda x: x['r.count'])
co_occurrence_data_before = [(f"{x['a.lemma']}, {x['b.lemma']}", x['r.count'] ) for x in results]
co_occurrence_data = []
while len(co_occurrence_data) < chart_length:
    item = co_occurrence_data_before.pop(0)
    if item[1]<10:
        continue
    if item[0] not in [x[0] for x in co_occurrence_data]+ ['',' ']:
        co_occurrence_data.append(item)

In [21]:
res = ""
for i  in range(chart_length):
    res+= f"{vision_level_data[i][0]} & {vision_level_data[i][1] :.0f} & {token_level_data[i][0]} & {token_level_data[i][1] :.0f} & {what_word_data[i][0]} & {what_word_data[i][1]:.0f} & {co_occurrence_data[i][0]}  & {co_occurrence_data[i][1] :.0f}  \\\\ \n"
    
    # res+= f"{vision_level_data[i][0]} & {vision_level_data[i][1]/1000 :.0f}K & {token_level_data[i][0]} & {token_level_data[i][1]/1000 :.0f}K & {what_word_data[i][0]} & {what_word_data[i][1]/1000:.0f}K & {co_occurrence_data[i][0]}  & {co_occurrence_data[i][1]/1000 :.0f}K   \\\\ \n"
    
print(res)

scissors one & 10 & ParkingIs & 10 & what animals & 10 & plane airplane, water lake  & 10  \\ 
father groom & 10 & ShoesWhich & 10 & what content & 10 & stovetop stove, window  & 10  \\ 
street highway road & 10 & BermudaWhat & 10 & be same & 10 & bowl dish, silverware  & 10  \\ 
eating meat & 10 & reagan & 10 & can you list & 10 & groom husband, tie  & 10  \\ 
plant palm tree & 10 & BambooWhat & 10 & what orientation & 10 & bat, road street  & 10  \\ 
woman grooming & 10 & polewhere & 10 & what purposes & 10 & net, one player  & 10  \\ 
man up & 10 & ballwhich & 10 & purpose & 10 & street, woman front  & 10  \\ 
cafe establishment & 10 & UmbrellaWhy & 10 & could you describe & 10 & attire clothing, bus  & 10  \\ 
colleague lady & 10 & snowis & 10 & what gesture & 10 & boat, lake waterway  & 10  \\ 
home museum & 10 & Crichton & 10 & does he have & 10 & microphone, suit attire  & 10  \\ 
instructor trainer & 10 & hourglass & 10 & what indication & 10 & glasses, suit attire  & 10  \\ 
o